In [3]:
import sklearn
import pandas as pd

import logging

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [4]:
train_df=pd.read_csv("..\..\..\Dataset\\bert\\train.csv")
eval_df=pd.read_csv("..\..\..\Dataset\\bert\\val.csv")


In [5]:
train_df=train_df
train_df

,sentence_id,words,labels
0,1382601382042103808,Hidup,O
1,1382601382042103808,sesedih,O
2,1382601382042103808,dan,O
3,1382601382042103808,secaper,O
4,1382601382042103808,apa,O
...,...,...,...
108840,1383795163412058114,Gubernur,O
108841,1383795163412058114,Khofifah,B-PER
108842,1383795163412058114,fokus,O
108843,1383795163412058114,untuk,O


In [6]:
eval_df

,sentence_id,words,labels
0,1385573262067847169,Nadiem,B-PER
1,1385573262067847169,Makarim,I-PER
2,1385573262067847169,dg,O
3,1385573262067847169,Presiden,O
4,1385573262067847169,ke-5,O
...,...,...,...
27206,1382913410749190149,aku,O
27207,1383795163412058114,ekonomi,O
27208,1383795163412058114,",",O
27209,1383795163412058114,infrastruktur,O


In [7]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [8]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    # "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    # "learning_rate": 0.00006828,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,
}


In [9]:
model = NERModel('bert', 
                'outputs',
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )
# result, model_outputs, wrong_preds = model.eval_model(eval_df)
# result

In [10]:
model = NERModel('bert', 
                'bert-base-cased',
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Downloading: 100%|██████████| 570/570 [00:00<00:00, 287kB/s]
Downloading: 100%|██████████| 436M/436M [01:05<00:00, 6.67MB/s]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

In [11]:
model.train_model(train_df,  eval_data=eval_df)

Epoch 1 of 1: 100%|██████████| 1/1 [07:19<00:00, 439.68s/it]


(722,
 {'global_step': [722],
  'precision': [0.6818858560794044],
  'recall': [0.5736951983298538],
  'f1_score': [0.6231292517006802],
  'train_loss': [0.21060845255851746],
  'eval_loss': [0.258757291057019]})

In [12]:
result, model_outputs, wrong_preds = model.eval_model(eval_df)

Running Evaluation: 100%|██████████| 269/269 [00:39<00:00,  6.85it/s]


In [12]:
result

{'eval_loss': 0.401481420783236,
 'precision': 0.46971153846153846,
 'recall': 0.40725302209253855,
 'f1_score': 0.4362580933244028}

In [12]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["Safari Ramadhan di Malang, Gubernur Khofifah ingatkan pentingnya Rembuk Nyekrup birokrat"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  3.12it/s]


In [11]:
predictions

NameError: name 'predictions' is not defined